## Importing the packages

In [1]:
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


## Initializing the model

In [2]:
model = Sequential()

## Loading the dataset

In [3]:
train_dataset = pd.read_csv("./data/train.csv")

In [4]:
test_dataset = pd.read_csv("./data/test.csv")

In [5]:
train_dataset.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test_dataset.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Splitting the dataset

In [7]:
X_data = train_dataset.iloc[:,1:].values
y_data = train_dataset.iloc[:,0].values

In [8]:
from sklearn.model_selection import train_test_split
X_data_train, X_data_test, y_train, y_test = train_test_split(X_data,y_data,test_size=.2,random_state=0)

## Reshaping the data into image-dimensions and Normalization

In [9]:
X_train = X_data_train.reshape(X_data_train.shape[0],28,28,1)
X_test = X_data_test.reshape(X_data_test.shape[0],28,28,1)

In [10]:
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

## One-hot encoding of the target

In [11]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,10)
y_test = np_utils.to_categorical(y_test,10)

## Model definition and training

In [12]:
model.add(Conv2D(32,(3,3), input_shape=(28,28,1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))
model.add(Conv2D(32,(3,3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))
model.add(Conv2D(64,(3,3)))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation("softmax"))

In [13]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [14]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=8,width_shift_range=0.08,shear_range=0.3,
                        height_shift_range=0.08,zoom_range=0.08)
test_gen = ImageDataGenerator()

In [15]:
train_generator = gen.flow(X_train,y_train,batch_size=64)
test_generator = test_gen.flow(X_test,y_test,batch_size=64)

In [16]:
model.fit_generator(train_generator,steps_per_epoch=360,epochs=7,
                   validation_data=test_generator,validation_steps=60)

Epoch 1/7
360/360 [==============================] - 299s 832ms/step - loss: 0.2087 - acc: 0.9357 - val_loss: 0.0676 - val_acc: 0.9792
Epoch 2/7
360/360 [==============================] - 303s 841ms/step - loss: 0.0793 - acc: 0.9755 - val_loss: 0.5887 - val_acc: 0.8698
Epoch 3/7
360/360 [==============================] - 305s 846ms/step - loss: 0.0699 - acc: 0.9782 - val_loss: 0.0412 - val_acc: 0.9859
Epoch 4/7
360/360 [==============================] - 318s 882ms/step - loss: 0.0515 - acc: 0.9839 - val_loss: 0.0615 - val_acc: 0.9784
Epoch 5/7
360/360 [==============================] - 356s 989ms/step - loss: 0.0487 - acc: 0.9857 - val_loss: 0.0306 - val_acc: 0.9924
Epoch 6/7
360/360 [==============================] - 308s 855ms/step - loss: 0.0471 - acc: 0.9849 - val_loss: 0.0296 - val_acc: 0.9885
Epoch 7/7
360/360 [==============================] - 293s 814ms/step - loss: 0.0449 - acc: 0.9854 - val_loss: 0.0231 - val_acc: 0.9911


## Saving the trained model for later use

In [17]:
model.save("DigitRecognizer.h5")

## Prediction on new data

In [23]:
test_data = test_dataset.values
test_data = test_data.reshape(test_data.shape[0],28,28,1)
test_data = test_data.astype("float32")
test_data /= 255

In [27]:
result = model.predict(test_data)

## Reverse one-hot encoding of the result

In [28]:
result.shape

(28000, 10)

In [39]:
import numpy as np
answer = np.argmax(result,axis=1)

## Saving the result in a csv file

In [42]:
result_df = pd.DataFrame({
    "ImageId":range(1,answer.shape[0]+1),
    "Label":answer
})

In [45]:
result_df.to_csv("result.csv",index=False)